In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pandas.core.dtypes.common import is_numeric_dtype

data = pd.read_csv("data.csv")
data = data.set_index('projectID')
data = data.drop(columns=['drives'])
data

,success,amount,female_creator,goal,words,backers_count,serial_entrepreneur,duration,category_art,category_comics,...,year_2020,year_2021,year_2022,media,sustainability,story,url,business,empathic,social
projectID,,,,,,,,,,,,,,,,,,,,,
3731940,0,203.0,0,6000.0,71,5,0,60.000000,0,0,...,0,0,0,0,0,app go benefit trainer personnel look get fit ...,https://www.kickstarter.com/projects/fitnesspo...,1.408,0.000,8.451
3451605,1,116639.0,0,10000.0,497,2003,1,14.583148,0,0,...,0,0,0,1,0,even miss kickstarter still get hand papillon ...,https://www.kickstarter.com/projects/kolossalg...,3.414,2.610,9.839
3722583,0,28.0,0,6000.0,443,6,0,30.000000,0,1,...,0,0,0,1,0,journey blak blak blak officially entitle blak...,https://www.kickstarter.com/projects/myheart/b...,0.903,0.677,13.318
3745862,1,1846.0,1,1500.0,474,48,0,39.687488,0,0,...,0,0,0,1,0,isabel jazz folk musical teach empathy metoo s...,https://www.kickstarter.com/projects/annawestb...,1.477,1.899,17.511
3735376,1,1001.0,1,1000.0,130,17,0,25.989560,0,0,...,0,0,0,0,0,main stage kids head new york young performer ...,https://www.kickstarter.com/projects/mainstage...,0.000,0.769,16.154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4378823,1,62669.0,0,28000.0,837,232,1,30.000000,0,0,...,0,0,1,1,0,book maui mauka makai companion book maui coas...,https://www.kickstarter.com/projects/danielsul...,0.239,1.912,6.810
4394019,1,11083.0,0,10000.0,167,80,0,26.377384,0,0,...,0,0,1,1,0,myrcene ale co dream mike goergen cy higgins m...,https://www.kickstarter.com/projects/myrceneal...,1.198,3.593,9.581
4385597,1,25265.0,0,25000.0,138,73,0,14.147211,0,0,...,0,0,1,1,0,firefly theatrical need help build theatre spa...,https://www.kickstarter.com/projects/fireflyth...,2.899,0.725,7.971


In [2]:
# Normalize data
# Box-Cox lambda values
# amount        0.0964858326046286
# goal          0.022711139651449448
# words         0.19404918878650212
# backers_count 0.010289931906306294
# duration      0.4346871019563134
# business      0.33987429799376034
# empathic      0.4179358821047982
# social        0.6219058616908149
# drives        0.22950948679659838 (old)

normData = data.copy()

for variable in ['amount', 'goal', 'words', 'backers_count']: # + drives
    normData[variable] = np.log10(normData[variable])
    normData = normData.rename(columns={variable: "norm_" + variable})

for variable in ['duration', 'business', 'empathic', 'social']:
    normData[variable] = np.sqrt(normData[variable])
    normData = normData.rename(columns={variable: 'norm_' + variable})

normData.to_csv('normalisedData.csv')
normData

,success,norm_amount,female_creator,norm_goal,norm_words,norm_backers_count,serial_entrepreneur,norm_duration,category_art,category_comics,...,year_2020,year_2021,year_2022,media,sustainability,story,url,norm_business,norm_empathic,norm_social
projectID,,,,,,,,,,,,,,,,,,,,,
3731940,0,2.307496,0,3.778151,1.851258,0.698970,0,7.745967,0,0,...,0,0,0,0,0,app go benefit trainer personnel look get fit ...,https://www.kickstarter.com/projects/fitnesspo...,1.186592,0.000000,2.907060
3451605,1,5.066844,0,4.000000,2.696356,3.301681,1,3.818789,0,0,...,0,0,0,1,0,even miss kickstarter still get hand papillon ...,https://www.kickstarter.com/projects/kolossalg...,1.847701,1.615549,3.136718
3722583,0,1.447158,0,3.778151,2.646404,0.778151,0,5.477226,0,1,...,0,0,0,1,0,journey blak blak blak officially entitle blak...,https://www.kickstarter.com/projects/myheart/b...,0.950263,0.822800,3.649384
3745862,1,3.266232,1,3.176091,2.675778,1.681241,0,6.299801,0,0,...,0,0,0,1,0,isabel jazz folk musical teach empathy metoo s...,https://www.kickstarter.com/projects/annawestb...,1.215319,1.378042,4.184615
3735376,1,3.000434,1,3.000000,2.113943,1.230449,0,5.097996,0,0,...,0,0,0,0,0,main stage kids head new york young performer ...,https://www.kickstarter.com/projects/mainstage...,0.000000,0.876926,4.019204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4378823,1,4.797053,0,4.447158,2.922725,2.365488,1,5.477226,0,0,...,0,0,1,1,0,book maui mauka makai companion book maui coas...,https://www.kickstarter.com/projects/danielsul...,0.488876,1.382751,2.609598
4394019,1,4.044657,0,4.000000,2.222716,1.903090,0,5.135892,0,0,...,0,0,1,1,0,myrcene ale co dream mike goergen cy higgins m...,https://www.kickstarter.com/projects/myrceneal...,1.094532,1.895521,3.095319
4385597,1,4.402519,0,4.397940,2.139879,1.863323,0,3.761278,0,0,...,0,0,1,1,0,firefly theatrical need help build theatre spa...,https://www.kickstarter.com/projects/fireflyth...,1.702645,0.851469,2.823296


In [3]:
decimals = 3

In [4]:
# Table 1
mData = data[data['female_creator'] == 0]
fData = data[data['female_creator'] == 1]

categories = ['category_art', 'category_comics', 'category_crafts', 'category_dance', 'category_design',
             'category_fashion', 'category_film&video', 'category_food', 'category_games', 'category_journalism',
             'category_music', 'category_photography', 'category_publishing', 'category_technology', 'category_theater']


def cleanPValue(pValue):
    if pValue < 0.001:
        return "< 0.001"
    else:
        return ('{:.%if}' % decimals).format(pValue)


mMeans = []
fMeans = []
tValues = []
pValuesTwo = []
pValuesOne = []
catNames = []
for category in categories:
    mCatData = mData[mData[category] == 1]
    fCatData = fData[fData[category] == 1]

    mMeans.append(('{:.%if}' % decimals).format(np.mean(mCatData['success'])))
    fMeans.append(('{:.%if}' % decimals).format(np.mean(fCatData['success'])))

    # Equal variance check
    mVar = np.var(mCatData['success'])
    fVar = np.var(fCatData['success'])

    bigger = np.maximum(mVar, fVar)
    smaller = np.minimum(mVar, fVar)

    equalVariance = False
    if bigger / smaller < 4:
        equalVariance = True

    # Perform two-sample t-test
    resultTwo = st.ttest_ind(a=fCatData['success'], b=mCatData['success'], equal_var=equalVariance)
    resultOne = st.ttest_ind(a=fCatData['success'], b=mCatData['success'], equal_var=equalVariance, alternative="greater")
    tValues.append(('{:.%if}' % decimals).format(resultTwo[0]))

    pValuesTwo.append(cleanPValue(resultTwo[1]))
    pValuesOne.append(cleanPValue(resultOne[1]))

    catNames.append(category.split('_')[1])

table1 = {"Category": catNames, "Female (mean)": fMeans, "Male (mean)": mMeans, "t-Value": tValues, "p-Value (two-sided)": pValuesTwo, "p-Value (one-sided)": pValuesOne}
table1 = pd.DataFrame.from_dict(table1)
table1 = table1.set_index('Category')
table1.to_csv('Tables/success rate per category.csv')
table1

,Female (mean),Male (mean),t-Value,p-Value (two-sided),p-Value (one-sided)
Category,,,,,
art,0.865,0.809,2.389,0.017,0.009
comics,0.884,0.934,-2.131,0.033,0.983
crafts,0.512,0.565,-0.661,0.509,0.745
dance,0.846,0.750,0.417,0.683,0.341
design,0.905,0.945,-1.804,0.072,0.964
fashion,0.809,0.578,5.433,< 0.001,< 0.001
film&video,0.623,0.542,1.959,0.051,0.025
food,0.422,0.482,-1.336,0.182,0.909
games,0.900,0.926,-1.300,0.194,0.903


In [5]:
# Table 2 (part of table 7)
# mVar = np.var(mData['success'])
# fVar = np.var(fData['success'])
#
# bigger = np.maximum(mVar, fVar)
# smaller = np.minimum(mVar, fVar)
#
# equalVariance = False
# if bigger / smaller < 4:
#     equalVariance = True
#
# result = st.ttest_ind(a=mData['success'], b=fData['success'], equal_var=equalVariance)
# if result[1] < 0.001:
#     result[1] = "< 0.001"
#
# table2 = {'Variable': ['success'], 'Male (mean)': [np.mean(mData['success'])], 'Female (mean)': [np.mean(fData['success'])], 't-Value': [result[0]], 'p-Value': [result[1]]}
# table2 = pd.DataFrame.from_dict(table2)
# table2 = table2.set_index('Variable')
# table2.to_csv('Tables/gender breakdown.csv')
# table2

In [6]:
# Table 3
numVars = []
means = []
medians = []
minima = []
maxima = []
SDs = []

for variable in data.columns.tolist():
    varData = data[variable]
    if is_numeric_dtype(varData):
        numVars.append(variable)
        means.append(('{:.%if}' % decimals).format(np.mean(varData)))
        medians.append(('{:.%if}' % decimals).format(np.median(varData)))
        minima.append(('{:.%if}' % decimals).format(np.min(varData)))
        maxima.append(('{:.%if}' % decimals).format(np.max(varData)))
        SDs.append(('{:.%if}' % decimals).format(np.std(varData)))

table3 = {"Variable": numVars, "Mean": means, "Median": medians, "Minimum": minima, "Maximum": maxima, "SD": SDs}
table3 = pd.DataFrame.from_dict(table3)
table3 = table3.set_index("Variable")
table3.to_csv('Tables/descriptive statistics.csv')
table3

,Mean,Median,Minimum,Maximum,SD
Variable,,,,,
success,0.763,1.000,0.000,1.000,0.425
amount,32911.323,3838.000,1.000,5988089.000,198177.017
female_creator,0.327,0.000,0.000,1.000,0.469
goal,19135.292,4000.000,1.000,14000000.000,182854.201
words,408.228,305.000,2.000,2588.000,362.641
backers_count,345.801,65.000,1.000,55024.000,1595.292
serial_entrepreneur,0.252,0.000,0.000,1.000,0.434
duration,32.190,30.000,1.000,73.958,12.615
category_art,0.121,0.000,0.000,1.000,0.327


In [7]:
# Table 4
succData = normData[normData['success'] == 1]
failData = normData[normData['success'] == 0]

succMeans = []
failMeans = []
succMedians = []
failMedians = []

meanTValues = []
meanPValuesTwo = []
meanPValuesOne = []

moodsVars = ['norm_amount', 'norm_goal', 'norm_words', 'norm_backers_count', 'norm_duration', 'norm_business', 'norm_empathic', 'norm_social'] # + norm_drives
medianTValues = []
medianPValues = []

numVars_ = []
for var in numVars:
    if var in ['amount', 'goal', 'words', 'backers_count', 'business', 'empathic', 'social', 'duration']:
        var = "norm_" + var
    numVars_.append(var)
numVars = numVars_[:]

for variable in numVars[1:]:
    varDataSucc = succData[variable]
    varDataFail = failData[variable]

    # Compute means and medians
    succMeans.append(('{:.%if}' % decimals).format(np.mean(varDataSucc)))
    failMeans.append(('{:.%if}' % decimals).format(np.mean(varDataFail)))
    succMedians.append(('{:.%if}' % decimals).format(np.median(varDataSucc)))
    failMedians.append(('{:.%if}' % decimals).format(np.median(varDataFail)))

    # Check variance ratio
    succVar = np.var(varDataSucc)
    failVar = np.var(varDataFail)

    bigger = np.maximum(succVar, failVar)
    smaller = np.minimum(succVar, failVar)

    equalVariance = False
    if bigger / smaller < 4:
        equalVariance = True

    # Perform two sample t-test
    resultTwo = st.ttest_ind(a=varDataSucc, b=varDataFail, equal_var=equalVariance)
    resultOne = st.ttest_ind(a=varDataSucc, b=varDataFail, equal_var=equalVariance, alternative='greater')
    meanTValues.append(('{:.%if}' % decimals).format(resultTwo[0]))

    meanPValuesTwo.append(cleanPValue(resultTwo[1]))
    meanPValuesOne.append(cleanPValue(resultOne[1]))

    # Perform Mood's median test
    if variable in moodsVars:
        result = st.median_test(varDataSucc, varDataFail)
        medianTValues.append(('{:.%if}' % decimals).format(result[0]))
        medianPValues.append(cleanPValue(result[1]))
    else:
        medianTValues.append('N/A')
        medianPValues.append('N/A')

table4 = {'Variable': numVars[1:], 'Success (mean)': succMeans, 'Failure (mean)': failMeans, 't-Value': meanTValues, 'p-Value (two-sided)': meanPValuesTwo, 'p-Value (one-sided)': meanPValuesOne}
table4 = pd.DataFrame.from_dict(table4)
table4 = table4.set_index('Variable')
table4.to_csv('Tables/mean per variable (success).csv')
table4

,Success (mean),Failure (mean),t-Value,p-Value (two-sided),p-Value (one-sided)
Variable,,,,,
norm_amount,3.833,2.064,79.058,< 0.001,< 0.001
female_creator,0.328,0.324,0.330,0.741,0.371
norm_goal,3.431,3.998,-31.352,< 0.001,1.000
norm_words,2.504,2.274,22.815,< 0.001,< 0.001
norm_backers_count,2.071,0.783,82.970,< 0.001,< 0.001
serial_entrepreneur,0.324,0.019,46.108,< 0.001,< 0.001
norm_duration,5.359,6.234,-32.865,< 0.001,1.000
category_art,0.134,0.080,6.466,< 0.001,< 0.001
category_comics,0.136,0.035,12.578,< 0.001,< 0.001


In [8]:
# Table 4 (continued)
table4 = {'Variable': numVars[1:], 'Success (median)': succMedians, 'Failure (median)': failMedians, 'χ²-Value': medianTValues, 'p-Value': medianPValues}
table4 = pd.DataFrame.from_dict(table4)
table4 = table4.set_index('Variable')
table4.to_csv('Tables/median per variable (success).csv')
table4

,Success (median),Failure (median),χ²-Value,p-Value
Variable,,,,
norm_amount,3.787,2.233,1736.985,< 0.001
female_creator,0.000,0.000,N/A,N/A
norm_goal,3.477,4.000,492.586,< 0.001
norm_words,2.538,2.291,373.029,< 0.001
norm_backers_count,2.017,0.699,2105.926,< 0.001
serial_entrepreneur,0.000,0.000,N/A,N/A
norm_duration,5.477,5.571,343.548,< 0.001
category_art,0.000,0.000,N/A,N/A
category_comics,0.000,0.000,N/A,N/A


In [9]:
# Table 5
table5 = {
    'money': ['cash', 'fines', 'invest', 'irs', 'market', 'profit', 'rich', 'spend', 'usd', 'worth'],
    'emotion': ['admire', 'clinical depression', 'envies', 'furious', 'heartbreaking', 'misery', 'not a good idea', 'rage', 'tantrum', 'you\'d love'],
    'social': ['admittedly', 'consoling', 'ex-wife', 'ladies man', 'mob', 'papa', 'stalking', 'transgender', 'village', 'white guy'],
    # 'drives': ['abuser', 'chat', 'dear', 'denies', 'excelling', 'law', 'mamacita', 'perfection', 'sergeant', 'sultan']
}
table5 = pd.DataFrame.from_dict(table5)
table5.to_csv('Tables/category examples.csv', index=False)
table5

,money,emotion,social
0,cash,admire,admittedly
1,fines,clinical depression,consoling
2,invest,envies,ex-wife
3,irs,furious,ladies man
4,market,heartbreaking,mob
5,profit,misery,papa
6,rich,not a good idea,stalking
7,spend,rage,transgender
8,usd,tantrum,village
9,worth,you'd love,white guy


In [10]:
# Table 7
mNormData = normData[normData['female_creator'] == 0]
fNormData = normData[normData['female_creator'] == 1]

mMeans = []
fMeans = []
tValues = []
pValuesTwo = []
pValuesOne = []

numVars_ = numVars[:]
numVars_.remove('female_creator')
for variable in numVars_:
    mVarData = mNormData[variable]
    fVarData = fNormData[variable]

    # Calculate the means
    mMeans.append(('{:.%if}' % decimals).format(np.mean(mVarData)))
    fMeans.append(('{:.%if}' % decimals).format(np.mean(fVarData)))

    # Check for equal variance
    mVar = np.var(mVarData)
    fVar = np.var(fVarData)

    bigger = np.maximum(mVar, fVar)
    smaller = np.minimum(mVar, fVar)

    equalVariance = False
    if bigger / smaller < 4:
        equalVariance = True

    # Perform two-sample t-test
    resultTwo = st.ttest_ind(a=fVarData, b=mVarData, equal_var=equalVariance)
    resultOne = st.ttest_ind(a=fVarData, b=mVarData, equal_var=equalVariance, alternative="greater")
    tValues.append(('{:.%if}' % decimals).format(resultTwo[0]))

    pValuesTwo.append(cleanPValue(resultTwo[1]))
    pValuesOne.append(cleanPValue(resultOne[1]))

table7 = {"Variable": numVars_, "Female (mean)": fMeans, "Male (mean)": mMeans, "t-Value": tValues, "p-Value (two-sided)": pValuesTwo, "p-Value (one-sided)": pValuesOne}
table7 = pd.DataFrame.from_dict(table7)
table7 = table7.set_index('Variable')
table7.to_csv('Tables/mean per variable (female_creator).csv')
table7

,Female (mean),Male (mean),t-Value,p-Value (two-sided),p-Value (one-sided)
Variable,,,,,
success,0.766,0.762,0.330,0.741,0.371
norm_amount,3.332,3.455,-4.601,< 0.001,1.000
norm_goal,3.468,3.612,-8.339,< 0.001,1.000
norm_words,2.421,2.464,-4.580,< 0.001,1.000
norm_backers_count,1.679,1.809,-6.889,< 0.001,1.000
serial_entrepreneur,0.199,0.278,-7.851,< 0.001,1.000
norm_duration,5.556,5.570,-0.541,0.588,0.706
category_art,0.234,0.067,22.725,< 0.001,< 0.001
category_comics,0.054,0.140,-11.902,< 0.001,1.000


In [11]:
# Determine which numerical variables are linearly related to amount
amountData = normData['norm_amount']

rValues = []
pValues = []
linears = []
for variable in numVars:
    rValue, pValue = st.pearsonr(amountData, normData[variable])
    rValues.append(rValue)
    pValues.append(pValue)
    linears.append(pValue < 0.05 and abs(rValue) >= 0.4)

correlations = pd.DataFrame()
correlations['Variable'] = numVars
correlations['r-Value'] = rValues
correlations['p-Value'] = pValues
correlations['Linear'] = linears
correlations = correlations.set_index('Variable')
correlations

,r-Value,p-Value,Linear
Variable,,,
success,0.653479,0.000000e+00,True
norm_amount,1.000000,0.000000e+00,True
female_creator,-0.050184,4.257402e-06,False
norm_goal,0.182873,5.444892e-64,False
norm_words,0.406861,0.000000e+00,True
norm_backers_count,0.911674,0.000000e+00,True
serial_entrepreneur,0.263469,3.246618e-133,False
norm_duration,-0.188017,1.337269e-67,False
category_art,-0.054990,4.669809e-07,False


In [12]:
# Establish a baseline model
independentVariables = numVars[:]
variablesToRemove = ['success', 'female_creator', 'norm_amount', 'norm_duration', 'norm_business', 'norm_empathic', 'norm_social']

for variable in independentVariables:
    for term in ['category_', 'month_', 'year_']:
        if term in variable:
            if variable not in variablesToRemove:
                variablesToRemove.append(variable)

for variable in variablesToRemove:
    independentVariables.remove(variable)

independentVariables += ['category_comics', 'category_fashion', 'year_2021']

X = sm.add_constant(normData[independentVariables])
y = normData['norm_amount']
baseModel = sm.OLS(y, X).fit()

baseModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            norm_amount   R-squared:                       0.847
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     5159.
Date:                Sun, 13 Nov 2022   Prob (F-statistic):               0.00
Time:                        19:52:23   Log-Likelihood:                -5198.7
No. Observations:                8388   AIC:                         1.042e+04
Df Residuals:                    8378   BIC:                         1.049e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.5170      0.037     13.918      0.000       0.444       0.590
norm_goal               0.1077      0.007     15.363      0.000       0.094       0.121
norm_words              0.0438      0.014      3.145      0.002       0.017       0.071
norm_backers_count      1.2402      0.008    165.002      0.000       1.225       1.255
serial_entrepreneur    -0.1131      0.013     -9.041      0.000      -0.138      -0.089
media                   0.2664      0.017     15.369      0.000       0.232       0.300
sustainability          0.1302      0.026      5.048      0.000       0.080       0.181
category_comics        -0.1429      0.016     -8.890      0.000      -0.174      -0.111
category_fashion       -0.0468      0.022     -2.090      0.037      -0.091      -0.003
year_2021               0.0740      0.010      7.078      0.000       0.053       0.094
==============================================================================
Omnibus:                      508.913   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1229.878
Skew:                          -0.369   Prob(JB):                    8.62e-268
Kurtosis:                       4.724   Cond. No.                         39.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
X = sm.add_constant(normData[independentVariables + ['norm_business']])
y = normData['norm_amount']
businessModel = sm.OLS(y, X).fit()

businessModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            norm_amount   R-squared:                       0.847
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     4649.
Date:                Sun, 13 Nov 2022   Prob (F-statistic):               0.00
Time:                        19:52:23   Log-Likelihood:                -5193.6
No. Observations:                8388   AIC:                         1.041e+04
Df Residuals:                    8377   BIC:                         1.049e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.5382      0.038     14.267      0.000       0.464       0.612
norm_goal               0.1080      0.007     15.409      0.000       0.094       0.122
norm_words              0.0490      0.014      3.494      0.000       0.022       0.076
norm_backers_count      1.2380      0.008    164.116      0.000       1.223       1.253
serial_entrepreneur    -0.1131      0.013     -9.044      0.000      -0.138      -0.089
media                   0.2644      0.017     15.252      0.000       0.230       0.298
sustainability          0.1326      0.026      5.144      0.000       0.082       0.183
category_comics        -0.1492      0.016     -9.217      0.000      -0.181      -0.117
category_fashion       -0.0442      0.022     -1.974      0.048      -0.088      -0.000
year_2021               0.0733      0.010      7.013      0.000       0.053       0.094
norm_business          -0.0224      0.007     -3.168      0.002      -0.036      -0.009
==============================================================================
Omnibus:                      510.537   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1235.446
Skew:                          -0.370   Prob(JB):                    5.32e-269
Kurtosis:                       4.728   Cond. No.                         40.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
X = sm.add_constant(normData[independentVariables + ['norm_empathic']])
y = normData['norm_amount']
empathicModel = sm.OLS(y, X).fit()

empathicModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            norm_amount   R-squared:                       0.847
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     4648.
Date:                Sun, 13 Nov 2022   Prob (F-statistic):               0.00
Time:                        19:52:23   Log-Likelihood:                -5194.7
No. Observations:                8388   AIC:                         1.041e+04
Df Residuals:                    8377   BIC:                         1.049e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.4868      0.039     12.598      0.000       0.411       0.563
norm_goal               0.1089      0.007     15.513      0.000       0.095       0.123
norm_words              0.0411      0.014      2.946      0.003       0.014       0.068
norm_backers_count      1.2407      0.008    165.090      0.000       1.226       1.255
serial_entrepreneur    -0.1103      0.013     -8.791      0.000      -0.135      -0.086
media                   0.2674      0.017     15.427      0.000       0.233       0.301
sustainability          0.1309      0.026      5.079      0.000       0.080       0.181
category_comics        -0.1430      0.016     -8.899      0.000      -0.174      -0.111
category_fashion       -0.0495      0.022     -2.209      0.027      -0.093      -0.006
year_2021               0.0731      0.010      7.000      0.000       0.053       0.094
norm_empathic           0.0221      0.008      2.820      0.005       0.007       0.037
==============================================================================
Omnibus:                      499.188   Durbin-Watson:                   1.948
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1215.812
Skew:                          -0.359   Prob(JB):                    9.77e-265
Kurtosis:                       4.721   Cond. No.                         41.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
X = sm.add_constant(normData[independentVariables + ['norm_social']])
y = normData['norm_amount']
socialModel = sm.OLS(y, X).fit()

socialModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            norm_amount   R-squared:                       0.847
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     4644.
Date:                Sun, 13 Nov 2022   Prob (F-statistic):               0.00
Time:                        19:52:23   Log-Likelihood:                -5197.8
No. Observations:                8388   AIC:                         1.042e+04
Df Residuals:                    8377   BIC:                         1.049e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.4901      0.042     11.614      0.000       0.407       0.573
norm_goal               0.1070      0.007     15.228      0.000       0.093       0.121
norm_words              0.0421      0.014      3.006      0.003       0.015       0.069
norm_backers_count      1.2413      0.008    164.146      0.000       1.226       1.256
serial_entrepreneur    -0.1114      0.013     -8.862      0.000      -0.136      -0.087
media                   0.2688      0.017     15.427      0.000       0.235       0.303
sustainability          0.1308      0.026      5.071      0.000       0.080       0.181
category_comics        -0.1443      0.016     -8.959      0.000      -0.176      -0.113
category_fashion       -0.0450      0.022     -2.006      0.045      -0.089      -0.001
year_2021               0.0739      0.010      7.070      0.000       0.053       0.094
norm_social             0.0090      0.007      1.343      0.179      -0.004       0.022
==============================================================================
Omnibus:                      506.461   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1225.944
Skew:                          -0.367   Prob(JB):                    6.16e-267
Kurtosis:                       4.723   Cond. No.                         52.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
X = sm.add_constant(normData[independentVariables + ['norm_business', 'norm_empathic', 'norm_social']])
y = normData['norm_amount']
combinedModel = sm.OLS(y, X).fit()

combinedModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            norm_amount   R-squared:                       0.847
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     3878.
Date:                Sun, 13 Nov 2022   Prob (F-statistic):               0.00
Time:                        19:52:23   Log-Likelihood:                -5189.8
No. Observations:                8388   AIC:                         1.041e+04
Df Residuals:                    8375   BIC:                         1.050e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.4942      0.043     11.516      0.000       0.410       0.578
norm_goal               0.1085      0.007     15.405      0.000       0.095       0.122
norm_words              0.0454      0.014      3.224      0.001       0.018       0.073
norm_backers_count      1.2393      0.008    163.434      0.000       1.224       1.254
serial_entrepreneur    -0.1095      0.013     -8.709      0.000      -0.134      -0.085
media                   0.2668      0.017     15.320      0.000       0.233       0.301
sustainability          0.1336      0.026      5.182      0.000       0.083       0.184
category_comics        -0.1500      0.016     -9.244      0.000      -0.182      -0.118
category_fashion       -0.0454      0.022     -2.019      0.044      -0.089      -0.001
year_2021               0.0725      0.010      6.943      0.000       0.052       0.093
norm_business          -0.0219      0.007     -3.073      0.002      -0.036      -0.008
norm_empathic           0.0188      0.008      2.307      0.021       0.003       0.035
norm_social             0.0059      0.007      0.849      0.396      -0.008       0.020
==============================================================================
Omnibus:                      500.672   Durbin-Watson:                   1.948
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1221.104
Skew:                          -0.360   Prob(JB):                    6.93e-266
Kurtosis:                       4.725   Cond. No.                         55.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
# Table 8


def denoteSignificanceLevel(value, pValue):
    value = ('{:.%if}' % decimals).format(value)

    if pValue <= 0.01:
        value += "***"
    elif pValue <= 0.05:
        value += "**"
    elif pValue <= 0.10:
        value += "*"

    return value


index = []
independentVariables = ['norm_business', 'norm_social', 'norm_empathic'] + independentVariables
for variable in independentVariables:
    index.append(variable)
    index.append('')
index += ['constant', 'Adj. R-squared', 'df']
table8 = pd.DataFrame(index=index)

models = [baseModel, businessModel, socialModel, empathicModel, combinedModel]
cntr = 1
for model in models:
    variables = model.params.index.tolist()

    data = []
    for variable in independentVariables:
        if variable in variables:
            varIndex = variables.index(variable)
            coeff = model.params.iloc[varIndex]
            pValue = model.pvalues.iloc[varIndex]
            data.append(denoteSignificanceLevel(coeff, pValue))
            if pValue < 0.001:
                pValue = '< 0.001'
            else:
                pValue = ('{:.%if}' % decimals).format(pValue)
            data.append('(%s)' % pValue)
        else:
            data.append('')
            data.append('')

    data.append(('{:.%if}' % decimals).format(model.params.iloc[0]))
    data.append(('{:.%if}' % decimals).format(model.rsquared_adj))
    data.append(('{:.%if}' % decimals).format(model.df_model))

    table8['Model ' + str(cntr)] = data
    cntr += 1

table8.to_csv('Tables/OLS models.csv')
table8

,Model 1,Model 2,Model 3,Model 4,Model 5
norm_business,,-0.022***,,,-0.022***
,,(0.002),,,(0.002)
norm_social,,,0.009,,0.006
,,,(0.179),,(0.396)
norm_empathic,,,,0.022***,0.019**
,,,,(0.005),(0.021)
norm_goal,0.108***,0.108***,0.107***,0.109***,0.109***
,(< 0.001),(< 0.001),(< 0.001),(< 0.001),(< 0.001)
norm_words,0.044***,0.049***,0.042***,0.041***,0.045***
,(0.002),(< 0.001),(0.003),(0.003),(0.001)


In [18]:
# Table 6
index = []
variables = ['norm_amount'] + independentVariables[:3] + ['female_creator'] + independentVariables[3:]
for variable in variables:
    index.append(variable)
    index.append('')
table6 = pd.DataFrame(index=index, columns=variables)

noVars = len(variables)
for i in range(noVars):
    rValues = []
    pValues = []

    var1Data = normData[variables[i]]
    for j in range(i + 1):
        if i == j:
            rValues.append(('{:.%if}' % decimals).format(1) + "***")
            pValues.append('(< 0.001)')
        else:
            rValue, pValue = st.pearsonr(var1Data, normData[variables[j]])
            rValues.append(denoteSignificanceLevel(rValue, pValue))

            if pValue < 0.001:
                pValue = '< 0.001'
            else:
                pValue = ('{:.%if}' % decimals).format(pValue)
            pValues.append('(%s)' % pValue)

    for j in range(i + 1, noVars):
        rValues.append('')
        pValues.append('')

    table6.iloc[2*i] = rValues
    table6.iloc[2*i + 1] = pValues

table6.to_csv('Tables/correlation matrix.csv')
table6

,norm_amount,norm_business,norm_social,norm_empathic,female_creator,norm_goal,norm_words,norm_backers_count,serial_entrepreneur,media,sustainability,category_comics,category_fashion,year_2021
norm_amount,1.000***,,,,,,,,,,,,,
,(< 0.001),,,,,,,,,,,,,
norm_business,-0.087***,1.000***,,,,,,,,,,,,
,(< 0.001),(< 0.001),,,,,,,,,,,,
norm_social,-0.136***,0.082***,1.000***,,,,,,,,,,,
,(< 0.001),(< 0.001),(< 0.001),,,,,,,,,,,
norm_empathic,-0.032***,-0.043***,0.275***,1.000***,,,,,,,,,,
,(0.004),(< 0.001),(< 0.001),(< 0.001),,,,,,,,,,
female_creator,-0.050***,0.121***,0.112***,0.122***,1.000***,,,,,,,,,
,(< 0.001),(< 0.001),(< 0.001),(< 0.001),(< 0.001),,,,,,,,,


In [19]:
formula = "norm_amount ~ norm_goal + norm_words + norm_backers_count + serial_entrepreneur + media + sustainability + category_comics + category_fashion + year_2021 + female_creator + norm_business + norm_business:female_creator"
businessModel = smf.ols(formula, data=normData).fit()

businessModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            norm_amount   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     3883.
Date:                Sun, 13 Nov 2022   Prob (F-statistic):               0.00
Time:                        19:52:23   Log-Likelihood:                -5184.9
No. Observations:                8388   AIC:                         1.040e+04
Df Residuals:                    8375   BIC:                         1.049e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.5174      0.038     13.455      0.000       0.442       0.593
norm_goal                        0.1114      0.007     15.795      0.000       0.098       0.125
norm_words                       0.0505      0.014      3.601      0.000       0.023       0.078
norm_backers_count               1.2379      0.008    164.214      0.000       1.223       1.253
serial_entrepreneur             -0.1098      0.013     -8.767      0.000      -0.134      -0.085
media                            0.2641      0.017     15.247      0.000       0.230       0.298
sustainability                   0.1275      0.026      4.944      0.000       0.077       0.178
category_comics                 -0.1430      0.016     -8.794      0.000      -0.175      -0.111
category_fashion                -0.0535      0.022     -2.383      0.017      -0.098      -0.009
year_2021                        0.0712      0.010      6.815      0.000       0.051       0.092
female_creator                   0.0263      0.023      1.152      0.249      -0.018       0.071
norm_business                   -0.0297      0.009     -3.439      0.001      -0.047      -0.013
norm_business:female_creator     0.0132      0.015      0.880      0.379      -0.016       0.043
==============================================================================
Omnibus:                      492.711   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1204.062
Skew:                          -0.353   Prob(JB):                    3.48e-262
Kurtosis:                       4.716   Cond. No.                         42.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
formula = "norm_amount ~ norm_goal + norm_words + norm_backers_count + serial_entrepreneur + media + sustainability + category_comics + category_fashion + year_2021 + female_creator + norm_empathic + norm_empathic:female_creator"
empathicModel = smf.ols(formula, data=normData).fit()

empathicModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            norm_amount   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     3879.
Date:                Sun, 13 Nov 2022   Prob (F-statistic):               0.00
Time:                        19:52:23   Log-Likelihood:                -5188.2
No. Observations:                8388   AIC:                         1.040e+04
Df Residuals:                    8375   BIC:                         1.049e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.4738      0.039     12.001      0.000       0.396       0.551
norm_goal                        0.1113      0.007     15.792      0.000       0.098       0.125
norm_words                       0.0425      0.014      3.043      0.002       0.015       0.070
norm_backers_count               1.2407      0.008    165.158      0.000       1.226       1.255
serial_entrepreneur             -0.1076      0.013     -8.570      0.000      -0.132      -0.083
media                            0.2676      0.017     15.451      0.000       0.234       0.302
sustainability                   0.1262      0.026      4.891      0.000       0.076       0.177
category_comics                 -0.1363      0.016     -8.429      0.000      -0.168      -0.105
category_fashion                -0.0558      0.022     -2.479      0.013      -0.100      -0.012
year_2021                        0.0718      0.010      6.870      0.000       0.051       0.092
female_creator                   0.0117      0.026      0.449      0.653      -0.039       0.063
norm_empathic                    0.0132      0.010      1.369      0.171      -0.006       0.032
norm_empathic:female_creator     0.0177      0.017      1.067      0.286      -0.015       0.050
==============================================================================
Omnibus:                      487.760   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1196.859
Skew:                          -0.348   Prob(JB):                    1.27e-260
Kurtosis:                       4.715   Cond. No.                         43.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
formula = "norm_amount ~ norm_goal + norm_words + norm_backers_count + serial_entrepreneur + media + sustainability + category_comics + category_fashion + year_2021 + female_creator + norm_social + norm_social:female_creator"
socialModel = smf.ols(formula, data=normData).fit()

socialModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            norm_amount   R-squared:                       0.847
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     3877.
Date:                Sun, 13 Nov 2022   Prob (F-statistic):               0.00
Time:                        19:52:23   Log-Likelihood:                -5190.6
No. Observations:                8388   AIC:                         1.041e+04
Df Residuals:                    8375   BIC:                         1.050e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.4922      0.045     10.963      0.000       0.404       0.580
norm_goal                      0.1096      0.007     15.499      0.000       0.096       0.123
norm_words                     0.0431      0.014      3.084      0.002       0.016       0.071
norm_backers_count             1.2410      0.008    164.191      0.000       1.226       1.256
serial_entrepreneur           -0.1088      0.013     -8.646      0.000      -0.133      -0.084
media                          0.2688      0.017     15.433      0.000       0.235       0.303
sustainability                 0.1258      0.026      4.874      0.000       0.075       0.176
category_comics               -0.1367      0.016     -8.424      0.000      -0.168      -0.105
category_fashion              -0.0516      0.023     -2.284      0.022      -0.096      -0.007
year_2021                      0.0722      0.010      6.904      0.000       0.052       0.093
female_creator                -0.0156      0.047     -0.329      0.742      -0.108       0.077
norm_social                    0.0007      0.008      0.085      0.932      -0.015       0.017
norm_social:female_creator     0.0164      0.014      1.180      0.238      -0.011       0.044
==============================================================================
Omnibus:                      493.560   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1203.167
Skew:                          -0.355   Prob(JB):                    5.44e-262
Kurtosis:                       4.714   Cond. No.                         67.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
formula = "norm_amount ~ norm_goal + norm_words + norm_backers_count + serial_entrepreneur + media + sustainability + category_comics + category_fashion + year_2021 + female_creator + norm_business + norm_business:female_creator + norm_empathic + norm_empathic:female_creator + norm_social + norm_social:female_creator"
combinedModel = smf.ols(formula, data=normData).fit()

combinedModel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            norm_amount   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     2914.
Date:                Sun, 13 Nov 2022   Prob (F-statistic):               0.00
Time:                        19:52:23   Log-Likelihood:                -5181.5
No. Observations:                8388   AIC:                         1.040e+04
Df Residuals:                    8371   BIC:                         1.052e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.5018      0.046     10.994      0.000       0.412       0.591
norm_goal                        0.1116      0.007     15.723      0.000       0.098       0.126
norm_words                       0.0480      0.014      3.406      0.001       0.020       0.076
norm_backers_count               1.2386      0.008    163.371      0.000       1.224       1.253
serial_entrepreneur             -0.1073      0.013     -8.521      0.000      -0.132      -0.083
media                            0.2663      0.017     15.287      0.000       0.232       0.300
sustainability                   0.1286      0.026      4.983      0.000       0.078       0.179
category_comics                 -0.1433      0.016     -8.771      0.000      -0.175      -0.111
category_fashion                -0.0522      0.023     -2.308      0.021      -0.097      -0.008
year_2021                        0.0708      0.010      6.774      0.000       0.050       0.091
female_creator                  -0.0328      0.052     -0.626      0.531      -0.136       0.070
norm_business                   -0.0290      0.009     -3.335      0.001      -0.046      -0.012
norm_business:female_creator     0.0145      0.015      0.962      0.336      -0.015       0.044
norm_empathic                    0.0114      0.010      1.137      0.255      -0.008       0.031
norm_empathic:female_creator     0.0146      0.017      0.845      0.398      -0.019       0.048
norm_social                      0.0004      0.008      0.046      0.963      -0.016       0.017
norm_social:female_creator       0.0100      0.014      0.690      0.490      -0.018       0.038
==============================================================================
Omnibus:                      487.353   Durbin-Watson:                   1.948
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1197.935
Skew:                          -0.347   Prob(JB):                    7.44e-261
Kurtosis:                       4.716   Cond. No.                         76.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
# Table 9
index = []
independentVariables = ['norm_business', 'norm_business:female_creator', 'norm_social', 'norm_social:female_creator', 'norm_empathic', 'norm_empathic:female_creator', 'female_creator'] + independentVariables[3:]
for variable in independentVariables:
    index.append(variable)
    index.append('')
index += ['constant', 'Adj. R-squared', 'df']
table9 = pd.DataFrame(index=index)

models = [baseModel, businessModel, socialModel, empathicModel, combinedModel]
cntr = 1
for model in models:
    variables = model.params.index.tolist()

    data = []
    for variable in independentVariables:
        if variable in variables:
            varIndex = variables.index(variable)
            coeff = model.params.iloc[varIndex]
            pValue = model.pvalues.iloc[varIndex]
            data.append(denoteSignificanceLevel(coeff, pValue))
            if pValue < 0.001:
                pValue = '< 0.001'
            else:
                pValue = ('{:.%if}' % decimals).format(pValue)
            data.append('(%s)' % pValue)
        else:
            data.append('')
            data.append('')

    data.append(('{:.%if}' % decimals).format(model.params.iloc[0]))
    data.append(('{:.%if}' % decimals).format(model.rsquared_adj))
    data.append(('{:.%if}' % decimals).format(model.df_model))

    table9['Model ' + str(cntr)] = data
    cntr += 1

table9.to_csv('Tables/OLS models (with interaction terms).csv')
table9

,Model 1,Model 2,Model 3,Model 4,Model 5
norm_business,,-0.030***,,,-0.029***
,,(< 0.001),,,(< 0.001)
norm_business:female_creator,,0.013,,,0.015
,,(0.379),,,(0.336)
norm_social,,,0.001,,0.000
,,,(0.932),,(0.963)
norm_social:female_creator,,,0.016,,0.010
,,,(0.238),,(0.490)
norm_empathic,,,,0.013,0.011
,,,,(0.171),(0.255)


In [24]:
# Robustness
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
def inCovidPeriod(project):
    if project['year_2020'] == 1:
        for month in months[2:]:
            if project['month_' + month] == 1:
                return True
    elif project['year_2021'] == 1:
        for month in months[:7]:
            if project['month_' + month] == 1:
                return True
    return False


def createModels(data_, subject):
    y_ = data_['norm_amount']
    models_ = []

    for additionVars in additionsVars:
        X_ = sm.add_constant(data_[baseVars + additionVars])
        models_.append(sm.OLS(y_, X_).fit())

    for additionFormula in additionsFormula:
        formula_ = baseFormula + additionFormula
        models_.append(smf.ols(formula_, data=data_).fit())

    table = pd.DataFrame(index=index)

    cntr_ = 1
    for model_ in models_:
        variables_ = model_.params.index.tolist()

        vals = []
        for variable_ in independentVariables:
            if variable_ in variables_:
                varIndex_ = variables_.index(variable_)
                coeff_ = model_.params.iloc[varIndex_]
                pValue_ = model_.pvalues.iloc[varIndex_]
                vals.append(denoteSignificanceLevel(coeff_, pValue_))
                if pValue_ < 0.001:
                    pValue_ = '< 0.001'
                else:
                    pValue_ = ('{:.%if}' % decimals).format(pValue_)
                vals.append('(%s)' % pValue_)
            else:
                vals.append('')
                vals.append('')

        vals.append(('{:.%if}' % decimals).format(model_.params.iloc[0]))
        vals.append(('{:.%if}' % decimals).format(model_.rsquared_adj))
        vals.append(('{:.%if}' % decimals).format(model_.df_model))

        table['Model ' + str(cntr_)] = vals
        cntr_ += 1

    table.to_csv('Tables/OLS models (%s).csv' % subject)


def compareLists(list1, list2):
    if len(list1) == len(list2):
        comparison = []
        for x in range(len(list1)):
            comparison.append(list1[x] or list2[x])
        return comparison
    else:
        return []


baseVars = independentVariables[7:]
additionsVars = [[], ['norm_business'], ['norm_social'], ['norm_empathic'], ['norm_business', 'norm_social', 'norm_empathic']]
baseFormula = "norm_amount ~ norm_goal + norm_words + norm_backers_count + serial_entrepreneur + media + sustainability + category_comics + category_fashion + year_2021 + female_creator + "
additionsFormula = ['norm_business + norm_business:female_creator', 'norm_social + norm_social:female_creator', 'norm_empathic + norm_empathic:female_creator', 'norm_business + norm_business:female_creator + norm_empathic + norm_empathic:female_creator + norm_social + norm_social:female_creator']

covidData = normData.copy()
covidData['COVID'] = covidData.apply(lambda project: inCovidPeriod(project), axis=1)
covidData = covidData[covidData['COVID']]
covidData = covidData.drop(columns=['COVID'])
createModels(covidData, 'COVID')

baseVars.remove('category_fashion')
baseFormula = baseFormula.replace(' category_fashion +', '')
fashionIndex = index.index('category_fashion')
del index[fashionIndex + 1]
del index[fashionIndex]
independentVariables.remove('category_fashion')
comicsFashionData = normData[compareLists((normData['category_comics'] == 1).tolist(), (normData['category_fashion'] == 1).tolist())]
createModels(comicsFashionData, 'comics & fashion')